# XQuery in Python: Process a whole collection at once on project XML files 

Source: <https://stackoverflow.com/questions/75142854/how-to-use-the-collection-function-with-saxonche> 

## New pip installs
* For this homework, you'll need to install the **pathlib** library so we can run XQuery across platforms on Mac *and* Windows. 
In your python environment for this series of notebooks, run `pip install pathlib`
* If you didn't install spacy before, be sure to `pip install spacy`
* Also, if you didn't download the spacy large language model, uncomment the line to download the spacy large model to your virtual environment.



In [1]:
# !pip install saxonche
# !pip install pathlib
import os
import pathlib
from pathlib import Path
import spacy
import re as regex
# re lets us work with regular expressions in Python
from saxonche import PySaxonProcessor
from os import getcwd
# this lets us retrieve the current working directory

Remember the spaCy language models? Let's try loading loading the large one to get the maximum amount of information from it! 
There's a lot we can experiment with from spaCy, so here's a link to the documentation for our ready reference:
<https://spacy.io/usage/spacy-101> 

We're going to start by just reviewing its POS (part of speech) and NER (named entity recognition) taggers to see what we can see in your project files.


In [2]:
# nlp = spacy.cli.download("en_core_web_lg")
# ONLY NEED ABOVE LINE ONCE. REMEMBER: COMMENT OUT THE ABOVE LINE THE NEXT TIME YOU RUN THIS.
nlp = spacy.load('en_core_web_lg')

Okay, let's explore some project files!
We've loaded the XML directory prepared by the Futurama team for our example here. 

* If you have some basic XML right now, like the Futurama team has prepared, we can easily scope in tagged sections of your collection. Swap out the Futurama collection with yours, and adjust the Python code below accordingly.
* If you don't have XML at this point, you can work around this over text files, or just explore the Futurama collection.

In [3]:
# DEFINE SOME FILE PATHS FOR INPUT, AND (ONCE WE'RE READY) OUTPUT
InputPath = 'futurama-xml'
OutputPath = 'testOutput' 

The next cell demonstrates the xpath() function, set up to run over individual files.
Let's look at how it returns information about distinct values of speakers. We're exploring distinct-values() and count() functions here. Try removing them and putting them back to see what the effect of distinct-values() is on the count. 


In [4]:
def readTextFiles(InputPath):
    # This function uses XPath to read the XML input
    for file in os.listdir(InputPath):
        if file.endswith('.xml'):
            filepath = f"{InputPath}/{file}"
            with PySaxonProcessor(license=False) as proc:
                xml = open(filepath, encoding='utf-8').read()
                # ebb: Here we apply the Saxon processor to read files with XPath.
                xp = proc.new_xpath_processor()
                node = proc.parse_xml(xml_text=xml)
                xp.set_context(xdm_item=node)

                # From here on, we select the string that Python will send to NLP. 
                # xpath = xp.evaluate('//your/xpath/here')
                xpath = xp.evaluate('//speak/@who => distinct-values() => sort()')
                count = xp.evaluate('//speak/@who => distinct-values() => count()')
                string = str(xpath)
                
                # xpath is going to go file by file.
             
readTextFiles(InputPath)

## Introducing XQuery! 
XQuery is what we want to use to help read data from across a whole directory, or "corpus" collection of files. 
XQuery can be written as a separate file (with .xql or .xquery extension) in oXygen over a directory. But we will find it more useful 
to apply it in Python if we're working on natural language processing applications. 

### Setting up XQuery in Python 
We use the same "boilerplate" PySaxonProcessor lines, but switch from xpath to the xquery processor.


Requirements: 
* We need all the xq lines to plug this processing into Python. You can use this code as a starter for your projects.
* The XQuery script is written inside a quoted block in the set_query_content() function that needs to take a quoted string, just like in the cell below.
* We need the run_query_to_value() function to execuit the script we're writing.
  

**Reading the collection**: We're setting this up to read a `collection()` function, which is a directory of XML files. 
**Writing XQuery**: This involves setting simple variables equal to xpath expressions. XQuery variables are defined with a `$`.
**XQuery Comments**: look like sideways smiley faces. `(: I'm an XQuery comment :)`

Let's take a look:



In [5]:
def xqueryOverFiles(InputPath):
    # This time, let's try XQuery over a collection of files:
    with PySaxonProcessor(license=False) as proc:
        print(proc.version)
        xq = proc.new_xquery_processor()
        # This only works on Mac / Linux: xq.set_query_base_uri('file://'+getcwd()+'/')
        xq.set_query_base_uri(Path('.').absolute().as_uri() + '/')
        xq.set_query_content('''
let $futurama := collection('futurama-xml/?select=*.xml')
let $speakers := $futurama//speak/@who => distinct-values() => sort()
let $count := count($speakers)
return $speakers

   (: ebb: I'm writing in an XQuery comment, and pointing out you can define and return any variable you want in the XQuery zone. :)
   (: ebb: Try changing this one to return $speakers instead of the $count variable. :)
   (: ebb: We're writing an query-based syntax called FLWOR (pronounced "flower") and every FLWOR requires a return statement at the end. :)
    
''')
        r = xq.run_query_to_value()
        print(r)  
                               
xqueryOverFiles(InputPath)

SaxonC-HE 12.4.2 from Saxonica
"5-EYES"
"AARON JR"
"AARON SR"
"AD ROCK"
"ADLAI"
"AHAB"
"AKI"
"ALBERT"
"ALBRIGHTBOT"
"ALEX"
"ALIEN"
"ALIENS"
"ALKAZAR"
"ALL"
"ALPHABOT"
"AMAZONIAN"
"AMAZONIANS"
"AMBASSADOR MOIVIN"
"AMY"
"AMY 1"
"AMY 420"
"ANDERSON"
"ANDREW"
"ANDY"
"ANGLEYNE"
"ANNOUNCER"
"ANNOUNCER #1"
"ANNOUNCER #2"
"ANTONIO"
"ARACHNEON"
"ARMY ROBOT"
"ARTHUR"
"ATILLA THE HUN"
"AUCTIONEER"
"AUDIENCE"
"AUSTRALIAN GUY"
"AUTOPILOT"
"BABE"
"BAILIFF"
"BAILLIF"
"BARBADOS SLIM"
"BARKER"
"BARMAN"
"BARRIERBOT #1"
"BARRIERBOT #2"
"BARTENDER"
"BARTENDERBOT"
"BEASTIE BOYS"
"BECK"
"BEE"
"BEELER"
"BEES"
"BENDER"
"BENDER 1"
"BENDER 1729"
"BENDER AND BECK"
"BENDER AND BENDER 1"
"BENDER FIGURINE"
"BENDING UNIT"
"BESERK"
"BETABOT"
"BETAMAX PLAYER"
"BIDDER #1"
"BIDDER #2"
"BIG BRAIN"
"BIG EARED MUTANT"
"BIG MOUTHED MUTANT"
"BIKE THIEF"
"BILL"
"BILLIONAIREBOT"
"BILLY"
"BLUE ELDER"
"BOLT"
"BONT"
"BOOTH VOICE"
"BOY"
"BRAIN #1"
"BRAIN #2"
"BRAIN #3"
"BRAIN BALL #1"
"BRAIN BALL #2"
"BRET"
"BROKERBOT #1"
"BROKERB

## What's the difference? 

Notice that you return just one count for the entire collection, not 72 different counts for the speakers in each file.
How can we use this? 

We can get pull information from across the entire collection and find out literally who has the most speeches in the whole series. 
Take a look at this code. Don't worry if you don't know how to write it yet. I just want to show you for demonstration purposes! 
What's here is a nearly full "FLWOR statement" which stands for :

* For
* Let
* Where
* Order by
* Return

The only things required in a FLWOR statements are L and R. The others give you extra powers like we're seeing here.

**IMPORTANT: You're only allowed one return per FLWOR.**



In [6]:
def xqueryOverFiles(InputPath):
    # This time, let's try XQuery over a collection of files:
    with PySaxonProcessor(license=False) as proc:
        print(proc.version)
        xq = proc.new_xquery_processor()
        xq.set_query_base_uri(Path('.').absolute().as_uri() + '/')
        xq.set_query_content('''
let $futurama := collection('futurama-xml/?select=*.xml')
let $speakers := $futurama//speak/@who => distinct-values() => sort()
let $count := count($speakers)
for $sp in $speakers
    let $count := $futurama//speak[@who = $sp] => count()
    where $count > 100
    order by $count descending
    return ($sp || ':  ' || $count)
 
''')
        r = xq.run_query_to_value()
        print(r)  
                            

xqueryOverFiles(InputPath)


SaxonC-HE 12.4.2 from Saxonica
"FRY:  2684"
"BENDER:  2360"
"LEELA:  2117"
"FARNSWORTH:  990"
"ZOIDBERG:  581"
"AMY:  499"
"HERMES:  408"
"ZAPP:  336"
"KIF:  197"
"CALCULON:  114"


For this notice how we can move **deliberately** in XQuery from information on the whole collection, to information based on individuals in a series.
The counts we're seeing are NOT based on files, but on info about each speaker! 

## For statements, measurements, order by, return


* Let's choose a character from Futurama who gets a LOT of speeches and use XQuery to pull all their speeches from across the entire collection into one return.
The next code block is written to show you how to get all the speeches of one character in the WHOLE collection. You can change this to any other character you wish.
* Let's go through each speech individually and meaasure it: 
* Try out defining a new variable in the XQuery, to use the `string-length()` function, so you can measure the text of the speeches. Try it out!

* We're going to write a for statement so we can evaluate each speech, and order our results based on how long the speeches are,
     * We'll use the XPath `string-length()` function to measure each speech
     * We'll work through an XQuery `for` and `order by`
     * Let's see if we can return the speeches from shortest to longest, then try adding the word `descending` to order them longest to shortest!


* YOUR TURN: Edit the code below to return some alternative information! 

In [21]:
def xqueryOverFiles(InputPath):
    
    with PySaxonProcessor(license=False) as proc:
        print(proc.version)
        xq = proc.new_xquery_processor()
        xq.set_query_base_uri(Path('.').absolute().as_uri() + '/')
        xq.set_query_content('''
let $futurama := collection('futurama-xml/?select=*.xml')
let $frySpeeches := $futurama//speak[@who="FRY"]
let $fryTextsOnly := $frySpeeches/text() 
let $frys := for $bt in $fryTextsOnly
   let $length := $bt ! string-length()
   where $length gt 20
   order by $length descending
   return $bt
   return string-join($frys, '\n\n')
''')

        r = xq.run_query_to_string()
        doc = nlp(r)
        for token in doc:
            print(token.text, token.head)

      
xqueryOverFiles(InputPath)

SaxonC-HE 12.4.2 from Saxonica
< version="1.0
? version="1.0
xml version="1.0
version="1.0 version="1.0
" version="1.0
encoding="UTF-8 version="1.0
" version="1.0
? version="1.0
> version="1.0

 >
Hell Hell
, Hell
no Hell
. Hell
If stopped
I stopped
stopped be
to think
think stopped
ahead think
, be

 ,
I be
would be
n't be
be be
Emperor be
. be
And be
I be
would be
n't be

 n't
even be
be be
here be
in here
the year
year in
3000 year
. be
It 's
's 's

 's
just like
like 's
the story
story like
of story
the grasshopper
grasshopper of

 grasshopper
and grasshopper
the octopus
octopus grasshopper
. 's
All year
year long
long kept
the grasshopper
grasshopper kept

 grasshopper
kept kept
burying kept
acorns burying
for burying
winter for
while mooched

 while
the octopus
octopus mooched
mooched burying
off mooched
his girlfriend
girlfriend mooched

 girlfriend
and mooched
watched mooched
TV watched
. kept
But came
then came
the winter
winter came

 winter
came came
and came
the grasshopper